# Ejercicio 04: Evaluación de un Sistema de Recuperación de Información



El objetivo de este ejercicio es evaluar la efectividad de un sistema de recuperación de información utilizando métricas como *precisión*, *recall*, *F1-score*, *Mean Average Precision (MAP)* y *Normalized Discounted Cumulative Gain (nDCG)*.



Seguirás los siguientes pasos:

Descripción del Ejercicio

1. Proporcionar un Conjunto de Datos:

    * Corpus de Documentos: Utiliza el corpus del ejercicio anterior o un nuevo conjunto de documentos.

    * Consultas: Define un conjunto de consultas específicas.

    * Juicios de Relevancia: Proporciona una lista de qué documentos son relevantes para cada consulta.

In [1]:
import xml.etree.ElementTree as ET
import math
from collections import defaultdict

In [2]:
# Paso 1: Proporcionar un Conjunto de Datos

# Corpus de Documentos: Utiliza el corpus del ejercicio anterior o un nuevo conjunto de documentos.

In [3]:
# Definir las consultas
queries = {
    1: "Impacto de la salud mental en el rendimiento académico de los estudiantes universitarios",
    2: "Actividades extracurriculares y bienestar emocional en el campus universitario",
    3: "Estrategias universitarias para reducir el estrés en estudiantes"
}

In [4]:
# Juicios de Relevancia: Proporciona una lista de qué documentos son relevantes para cada consulta.
# Los juicios de relevancia son las listas de documentos que consideramos relevantes para cada consulta.
relevant_docs = {
    1: {1, 2, 7},  # Relevantes para la consulta 1
    2: {5, 8, 15},  # Relevantes para la consulta 2
    3: {4, 7, 9}    # Relevantes para la consulta 3
}

2. Calcular Resultados de Búsqueda:

    * Obten los resultados ordenados de dos sistemas de recuperación para cada consulta.

In [5]:
# Paso 2: Calcular Resultados de Búsqueda

# Función para procesar el texto y extraer palabras clave
def process_text(text):
    text = text.lower()  # Convertir todo el texto a minúsculas para normalizar
    import re
    text = re.sub(r'[^a-záéíóúñü]+', ' ', text)  # Eliminar caracteres no alfanuméricos
    tokens = text.strip().split()  # Tokenizar el texto en palabras
    return set(tokens)  # Devolver un conjunto de palabras clave


In [6]:
# Leer y parsear el archivo XML del corpus de documentos
def parse_corpus(xml_file):
    tree = ET.parse(xml_file)  # Parsear el archivo XML
    root = tree.getroot()  # Obtener el nodo raíz del archivo XML
    corpus = {}  # Diccionario para almacenar los documentos procesados
    
    for doc in root.findall('document'):  # Iterar sobre cada documento en el corpus
        doc_id = int(doc.get('id'))  # Obtener el ID del documento
        title = doc.find('title').text  # Obtener el título del documento
        keywords = doc.find('keywords').text  # Obtener las palabras clave del documento
        author = doc.find('author').text  # Obtener el autor del documento
        date = doc.find('date').text  # Obtener la fecha de publicación del documento
        
        # Procesar las palabras clave
        keyword_set = process_text(keywords)
        
        # Almacenar el documento en el diccionario 'corpus'
        corpus[doc_id] = {
            'title': title,
            'keywords': keyword_set,
            'author': author,
            'date': date
        }
    
    return corpus  # Devolver el diccionario con los documentos procesados

In [7]:
# Calcular la similitud de Jaccard
def jaccard_similarity(set1, set2):
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return intersection / union if union != 0 else 0  

In [8]:
# Calcular la similitud de Coseno
def cosine_similarity(set1, set2):
    intersection = len(set1.intersection(set2)) 
    return intersection / (math.sqrt(len(set1)) * math.sqrt(len(set2))) if len(set1) > 0 and len(set2) > 0 else 0  # Calcular y devolver la similitud coseno


In [9]:
# Función para obtener los resultados ordenados de búsqueda
def get_search_results(queries, corpus, similarity_func):
    results = {}  # Diccionario para almacenar los resultados de búsqueda
    
    # Iterar sobre cada consulta
    for query_id, query in queries.items():
        query_set = process_text(query)  # Procesar la consulta
        similarities = []  # Lista para almacenar la similitud con cada documento
        
        # Comparar la consulta con cada documento
        for doc_id, doc in corpus.items():
            doc_set = doc['keywords']  # Obtener las palabras clave del documento
            similarity = similarity_func(query_set, doc_set)  # Calcular la similitud
            similarities.append((doc_id, similarity))  # Almacenar el ID del documento y la similitud
        
        # Ordenar los documentos por similitud (de mayor a menor)
        similarities.sort(key=lambda x: x[1], reverse=True)
        results[query_id] = similarities  # Almacenar los resultados ordenados de búsqueda
    
    return results  # Devolver los resultados

3. Calcular las Métricas de Evaluación:

    * Calcular las siguientes métricas para cada sistema y consulta:

        * Precisión en el top-k (Prec@k)

        * Recall

        * F1-score

        * Mean Average Precision (MAP)

        * nDCG

In [10]:
# Paso 3: Calcular las Métricas de Evaluación

# Calcular la precisión en el top-k
def precision_at_k(results, relevant_docs, k):
    precision = {}  # Diccionario para almacenar la precisión para cada consulta
    for query_id, docs in results.items():
        retrieved = [doc_id for doc_id, _ in docs[:k]]  # Obtener los primeros k documentos recuperados
        relevant = relevant_docs[query_id]  # Obtener los documentos relevantes para la consulta
        relevant_retrieved = len(set(retrieved).intersection(relevant))  # Contar cuántos documentos relevantes fueron recuperados
        precision[query_id] = relevant_retrieved / k  # Calcular la precisión y almacenarla
    return precision

In [11]:
# Calcular el recall
def recall(results, relevant_docs, k):
    recall_scores = {}  # Diccionario para almacenar el recall para cada consulta
    for query_id, docs in results.items():
        retrieved = [doc_id for doc_id, _ in docs[:k]]  # Obtener los primeros k documentos recuperados
        relevant = relevant_docs[query_id]  # Obtener los documentos relevantes para la consulta
        relevant_retrieved = len(set(retrieved).intersection(relevant))  # Contar cuántos documentos relevantes fueron recuperados
        recall_scores[query_id] = relevant_retrieved / len(relevant)  # Calcular el recall y almacenarlo
    return recall_scores

In [12]:
# Calcular el F1-score
def f1_score(precision, recall):
    f1_scores = {}  # Diccionario para almacenar el F1-score
    for query_id in precision.keys():
        p = precision[query_id]  # Precisión para la consulta
        r = recall[query_id]  # Recall para la consulta
        f1_scores[query_id] = (2 * p * r) / (p + r) if (p + r) != 0 else 0  # Calcular el F1-score
    return f1_scores

In [13]:
def mean_average_precision(results, relevant_docs):
    map_score = 0  # Inicializar el MAP en 0
    for query_id, docs in results.items():
        relevant_retrieved = 0  # Contador de documentos relevantes recuperados
        average_precision = 0  # Inicializar la precisión promedio para esta consulta
        for i, (doc_id, _) in enumerate(docs):
            if doc_id in relevant_docs[query_id]:  # Si el documento es relevante
                relevant_retrieved += 1  # Incrementar el contador
                # Calcular la precisión en la posición i (i+1 porque las posiciones empiezan desde 0)
                average_precision += relevant_retrieved / (i + 1)
        # Si hay documentos relevantes, calcular la precisión promedio
        if relevant_retrieved > 0:
            average_precision /= relevant_retrieved
        map_score += average_precision  # Sumar la precisión promedio de esta consulta

    # Dividir entre el número de consultas para obtener el MAP final
    return map_score / len(results)  # MAP promedio para todas las consultas


In [14]:
# Calcular el nDCG (Normalized Discounted Cumulative Gain)
def ndcg(results, relevant_docs, k):
    def dcg_at_k(docs, relevant):
        dcg = 0  # Inicializar el DCG
        for i, (doc_id, _) in enumerate(docs[:k]):  # Iterar sobre los primeros k documentos
            if doc_id in relevant:  # Si el documento es relevante
                dcg += 1 / math.log2(i + 2)  # Calcular el DCG con descuento
        return dcg

    def ideal_dcg(relevant):
        ideal_relevance = sorted(relevant, reverse=True)  # Relevancia ideal (ordenada de mayor a menor)
        return dcg_at_k([(doc_id, 1) for doc_id in ideal_relevance], relevant)  # Calcular el DCG ideal
    
    ndcg_scores = {}  # Diccionario para almacenar el nDCG
    for query_id, docs in results.items():
        dcg = dcg_at_k(docs, relevant_docs[query_id])  # Calcular el DCG de los resultados recuperados
        ideal = ideal_dcg(relevant_docs[query_id])  # Calcular el DCG ideal
        ndcg_scores[query_id] = dcg / ideal if ideal > 0 else 0  # Calcular el nDCG y almacenarlo
    return ndcg_scores

4. Análisis y Comparación:

    * Comparar los resultados de los dos sistemas utilizando las métricas calculadas.

    * Discutir cuál sistema es más efectivo y por qué.

In [15]:
# Paso 4: Análisis y Comparación

# Obtener los resultados utilizando Jaccard y Coseno
corpus = parse_corpus('../data/03ranking_corpus.xml')  
jaccard_results = get_search_results(queries, corpus, jaccard_similarity)  # Resultados con Jaccard
cosine_results = get_search_results(queries, corpus, cosine_similarity)  # Resultados con Coseno


In [16]:
# Calcular las métricas para ambos métodos (Jaccard y Coseno)
k = 5  # Consideramos el top-5
precision_jaccard = precision_at_k(jaccard_results, relevant_docs, k)
recall_jaccard = recall(jaccard_results, relevant_docs, k)
f1_jaccard = f1_score(precision_jaccard, recall_jaccard)
map_jaccard = mean_average_precision(jaccard_results, relevant_docs)
ndcg_jaccard = ndcg(jaccard_results, relevant_docs, k)

precision_cosine = precision_at_k(cosine_results, relevant_docs, k)
recall_cosine = recall(cosine_results, relevant_docs, k)
f1_cosine = f1_score(precision_cosine, recall_cosine)
map_cosine = mean_average_precision(cosine_results, relevant_docs)
ndcg_cosine = ndcg(cosine_results, relevant_docs, k)

In [17]:
# Función para calcular el promedio de las métricas
def calculate_average(metrics):
    return sum(metrics.values()) / len(metrics)

In [18]:
# Mostrar las métricas por cada consulta y los promedios
print("\nMétricas para Jaccard:")
for query_id in queries:
    print(f"Consulta {query_id}:")
    print(f"  Precisión: {precision_jaccard[query_id]}")
    print(f"  Recall: {recall_jaccard[query_id]}")
    print(f"  F1-score: {f1_jaccard[query_id]}")
    print(f"  MAP: {map_jaccard}")
    print(f"  nDCG: {ndcg_jaccard[query_id]}")
    print()


Métricas para Jaccard:
Consulta 1:
  Precisión: 0.4
  Recall: 0.6666666666666666
  F1-score: 0.5
  MAP: 0.5687533078837427
  nDCG: 0.7653606369886217

Consulta 2:
  Precisión: 0.6
  Recall: 1.0
  F1-score: 0.7499999999999999
  MAP: 0.5687533078837427
  nDCG: 0.9060254355346823

Consulta 3:
  Precisión: 0.2
  Recall: 0.3333333333333333
  F1-score: 0.25
  MAP: 0.5687533078837427
  nDCG: 0.20210734650054757



In [19]:
# Calcular y mostrar el promedio de cada métrica para Jaccard
print("Promedio de Métricas para Jaccard:")
print(f"  Promedio Precisión: {calculate_average(precision_jaccard)}")
print(f"  Promedio Recall: {calculate_average(recall_jaccard)}")
print(f"  Promedio F1-score: {calculate_average(f1_jaccard)}")
print(f"  Promedio MAP: {map_jaccard}")
print(f"  Promedio nDCG: {calculate_average(ndcg_jaccard)}")
print()

Promedio de Métricas para Jaccard:
  Promedio Precisión: 0.39999999999999997
  Promedio Recall: 0.6666666666666666
  Promedio F1-score: 0.5
  Promedio MAP: 0.5687533078837427
  Promedio nDCG: 0.6244978063412838



In [20]:
# Mostrar las métricas para Coseno
print("\nMétricas para Coseno:")
for query_id in queries:
    print(f"Consulta {query_id}:")
    print(f"  Precisión: {precision_cosine[query_id]}")
    print(f"  Recall: {recall_cosine[query_id]}")
    print(f"  F1-score: {f1_cosine[query_id]}")
    print(f"  MAP: {map_cosine}")
    print(f"  nDCG: {ndcg_cosine[query_id]}")
    print()


Métricas para Coseno:
Consulta 1:
  Precisión: 0.4
  Recall: 0.6666666666666666
  F1-score: 0.5
  MAP: 0.5687533078837427
  nDCG: 0.7653606369886217

Consulta 2:
  Precisión: 0.6
  Recall: 1.0
  F1-score: 0.7499999999999999
  MAP: 0.5687533078837427
  nDCG: 0.9060254355346823

Consulta 3:
  Precisión: 0.2
  Recall: 0.3333333333333333
  F1-score: 0.25
  MAP: 0.5687533078837427
  nDCG: 0.20210734650054757



In [21]:
# Calcular y mostrar el promedio de cada métrica para Coseno
print("Promedio de Métricas para Coseno:")
print(f"  Promedio Precisión: {calculate_average(precision_cosine)}")
print(f"  Promedio Recall: {calculate_average(recall_cosine)}")
print(f"  Promedio F1-score: {calculate_average(f1_cosine)}")
print(f"  Promedio MAP: {map_cosine}")
print(f"  Promedio nDCG: {calculate_average(ndcg_cosine)}")


Promedio de Métricas para Coseno:
  Promedio Precisión: 0.39999999999999997
  Promedio Recall: 0.6666666666666666
  Promedio F1-score: 0.5
  Promedio MAP: 0.5687533078837427
  Promedio nDCG: 0.6244978063412838
